In [4]:
from reaktoro import *

In [5]:
db = SupcrtDatabase("supcrtbl")

In [6]:
solution = AqueousPhase(speciate("Na Cl C Ca Mg Si"), exclude("organic"))
solution.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2")
))

In [7]:
gases = GaseousPhase("CO2(g) H2O(g)")
gases.setActivityModel(ActivityModelPengRobinson())

In [8]:
minerals = MineralPhases("Halite Calcite Magnesite Dolomite Quartz")

In [9]:
system = ChemicalSystem(db, solution, gases, minerals)

In [10]:
statex = ChemicalState(system)
statex.temperature(60.0, "celsius")
statex.pressure(100.0, "bar")
statex.set("H2O(aq)"  , 1.00, "kg")
statex.set("Halite"   , 1.00, "mol")
statex.set("Calcite"  , 1.00, "mol")
statex.set("Magnesite", 1.00, "mol")
statex.set("Quartz"   , 1.00, "mol")

In [11]:
equilibrate(statex)

In [12]:
propsx = ChemicalProps(statex)
Vx = propsx.volume()
Ux = propsx.internalEnergy()

In [13]:
statex.output("state-expected.txt")
propsx.output("props-expected.txt")

In [14]:
specs = EquilibriumSpecs(system)

In [15]:
idxV = specs.addInput("V")
idxU = specs.addInput("U")

In [16]:
volumeConstraint = ConstraintEquation()
volumeConstraint.id = "VolumeConstraint"
volumeConstraint.fn = lambda props, w: props.volume() - w[idxV]

In [17]:
internalEnergyConstraint = ConstraintEquation()
internalEnergyConstraint.id = "InternalEnergyConstraint"
internalEnergyConstraint.fn = lambda props, w: props.internalEnergy() - w[idxU]

In [18]:
specs.addConstraint(volumeConstraint)
specs.addConstraint(internalEnergyConstraint)

In [19]:
conditions = EquilibriumConditions(specs)
conditions.set("V", Vx)
conditions.set("U", Ux)
conditions.setLowerBoundPressure(1.0, "bar")

In [20]:
state = ChemicalState(system)
state.temperature(25.0, "celsius")
state.pressure(1.0, "bar")
state.set("H2O(aq)"  , 1.00, "kg")
state.set("Halite"   , 1.00, "mol")
state.set("Calcite"  , 1.00, "mol")
state.set("Magnesite", 1.00, "mol")
state.set("Quartz"   , 1.00, "mol")

In [21]:
solver = EquilibriumSolver(specs)

In [22]:
solver.solve(state, conditions)

In [23]:
props = ChemicalProps(state)

In [24]:
state.output("state.txt")
props.output("props.txt")

Check if props.txt and props-expected.txt are numerically equivalent.